# Imports

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error


2024-05-31 18:46:01.399377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 18:46:01.399506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 18:46:01.553504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Exploration

In [2]:
# Load the datasets
train_data = pd.read_csv("/kaggle/input/widsdatathon2024-challenge2/train.csv")
test_data = pd.read_csv("/kaggle/input/widsdatathon2024-challenge2/test.csv")

In [3]:
# Data exploration
print(train_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13173 entries, 0 to 13172
Columns: 152 entries, patient_id to metastatic_diagnosis_period
dtypes: float64(137), int64(4), object(11)
memory usage: 15.3+ MB
None


In [4]:
# Display the dataframe in a tabular format
display(HTML(train_data.head().to_html()))


,patient_id,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,metastatic_first_novel_treatment,metastatic_first_novel_treatment_type,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_75_to_100,income_household_100_to_150,income_household_150_over,income_household_six_figure,income_individual_median,home_ownership,housing_units,home_value,rent_median,rent_burden,education_less_highschool,education_highschool,education_some_college,education_bachelors,education_graduate,education_college_or_above,education_stem_degree,labor_force_participation,unemployment_rate,self_employed,farmer,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Average of Jan-13,Average of Feb-13,Average of Mar-13,Average of Apr-13,Average of May-13,Average of Jun-13,Average of Jul-13,Average of Aug-13,Average of Sep-13,Average of Oct-13,Average of Nov-13,Average of Dec-13,Average of Jan-14,Average of Feb-14,Average of Mar-14,Average of Apr-14,Average of May-14,Average of Jun-14,Average of Jul-14,Average of Aug-14,Average of Sep-14,Average of Oct-14,Average of Nov-14,Average of Dec-14,Average of Jan-15,Average of Feb-15,Average of Mar-15,Average of Apr-15,Average of May-15,Average of Jun-15,Average of Jul-15,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
0,268700,NaN,COMMERCIAL,AR,724,South,West South Central,39,F,NaN,C50912,Malignant neoplasm of unspecified site of left female breast,C773,NaN,NaN,3924.87,82.63,42.58,11.61,13.03,10.87,11.80,12.29,13.22,13.47,10.07,3.64,51.43,48.57,51.05,16.72,23.57,8.66,3.01,43.99,44483.35,2.21,3.97,8.52,7.08,7.67,13.82,15.14,17.51,11.26,8.90,3.93,12.83,24048.55,72.11,1513.75,87384.33,641.39,27.52,16.55,41.83,28.31,9.21,4.11,13.32,38.78,53.60,5.85,11.82,5.31,92.95,1.73,0.33,0.20,0.03,0.83,3.94,3.03,22.24,19.27,0.42,25.35,8.06,8.11,38.55,39.88,42.75,55.16,65.17,75.98,76.75,76.45,73.67,59.73,45.18,37.43,31.67,33.83,42.35,57.72,67.35,75.92,74.28,79.59,70.84,62.39,41.89,41.46,35.24,31.10,45.50,60.94,68.49,77.69,81.35,76.28,73.32,60.32,51.00,48.37,35.76,42.89,53.21,61.07,66.29,82.78,81.52,79.52,75.61,65.76,53.85,39.56,41.59,49.01,52.28,63.40,67.11,75.78,80.58,76.16,72.91,62.68,49.73,39.80,31.71,41.69,50.22,52.55,74.77,79.96,81.69,78.30,74.56,59.98,42.98,41.18,191
1,484983,White,NaN,IL,629,Midwest,East North Central,55,F,35.36,C50412,Malig neoplasm of upper-outer quadrant of left female breast,C773,NaN,NaN,2745.39,51.79,43.54,11.22,12.19,11.45,11.01,11.35,14.39,14.15,9.17,5.05,49.32,50.68,49.48,15.42,26.93,8.17,3.17,41.41,51796.79,3.67,3.86,6.58,5.58,5.38,11.02,13.09,19.56,11.76,11.40,8.11,19.51,28028.04,76.71,1113.35,92026.84,638.60,29.37,10.93,35.26,35.33,12.46,6.04,18.49,36.35,52.51,7.45,9

In [5]:
print(train_data.isnull().sum())

patient_id                        0
patient_race                   6657
payer_type                     1765
patient_state                     0
patient_zip3                      0
                               ... 
Average of Sep-18                 7
Average of Oct-18                 7
Average of Nov-18                12
Average of Dec-18                33
metastatic_diagnosis_period       0
Length: 152, dtype: int64


In [6]:
# Function to calculate percentage of missing values in each column
def missing_percentage(df):
    missing_values_percentage = (df.isnull().sum() / len(df)) * 100
    return missing_values_percentage


In [7]:
# Adjust display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [8]:
# Print percentage of missing values in each column
missing_percentages = missing_percentage(train_data)
print("Percentage of missing values in train dataset:")
print(missing_percentages)

Percentage of missing values in train dataset:
patient_id                                0.000000
patient_race                             50.535186
payer_type                               13.398618
patient_state                             0.000000
patient_zip3                              0.000000
Region                                    0.000000
Division                                  0.000000
patient_age                               0.000000
patient_gender                            0.000000
bmi                                      68.860548
breast_cancer_diagnosis_code              0.000000
breast_cancer_diagnosis_desc              0.000000
metastatic_cancer_diagnosis_code          0.000000
metastatic_first_novel_treatment         99.916496
metastatic_first_novel_treatment_type    99.916496
population                                0.000000
density                                   0.000000
age_median                                0.000000
age_under_10                       

In [9]:
# Print percentage of missing values in each column
missing_percentages = missing_percentage(test_data)
print("Percentage of missing values in test dataset:")
print(missing_percentages)

Percentage of missing values in test dataset:
patient_id                                0.000000
patient_race                             49.326957
payer_type                               13.903649
patient_state                             0.000000
patient_zip3                              0.000000
Region                                    0.000000
Division                                  0.000000
patient_age                               0.000000
patient_gender                            0.000000
bmi                                      69.801629
breast_cancer_diagnosis_code              0.000000
breast_cancer_diagnosis_desc              0.000000
metastatic_cancer_diagnosis_code          0.000000
metastatic_first_novel_treatment         99.876018
metastatic_first_novel_treatment_type    99.876018
population                                0.000000
density                                   0.000000
age_median                                0.000000
age_under_10                        

# Data Cleaning and Data preprocessing

In [10]:
# Drop columns with more than 50% missing values
train_data = train_data.dropna(thresh=len(train_data)*0.5, axis=1)
test_data = test_data.dropna(thresh=len(test_data)*0.5, axis=1)



In [11]:
# Drop the target variable from the train data
X = train_data.drop(columns=['metastatic_diagnosis_period'])  
y = train_data['metastatic_diagnosis_period']



In [12]:
# Select only numeric columns for imputation
numeric_columns = X.select_dtypes(include=np.number).columns.tolist()

# Make a copy of numeric_columns for imputation
numeric_columns_impute = numeric_columns.copy()

# Impute missing values with mean for numeric columns in train data
imputer_train = SimpleImputer(strategy='mean')
X[numeric_columns_impute] = imputer_train.fit_transform(X[numeric_columns_impute])

# Impute missing values with mean for numeric columns in test data
test_data[numeric_columns_impute] = imputer_train.transform(test_data[numeric_columns_impute])



In [13]:
# Impute missing values with mode for non-numeric columns
categorical_columns = X.select_dtypes(exclude=np.number).columns.tolist()
X[categorical_columns] = X[categorical_columns].fillna(X.mode().iloc[0])
test_data[categorical_columns] = test_data[categorical_columns].fillna(test_data.mode().iloc[0])



In [14]:
# Encode categorical variables
X = pd.get_dummies(X)
test_data = pd.get_dummies(test_data)



In [15]:
# Align train and test data to make sure they have the same columns
X, test_data = X.align(test_data, join='inner', axis=1)

# Model Building and Training

In [16]:
# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Convert 'patient_id' column to integer type
test_data['patient_id'] = test_data['patient_id'].astype(int)

# Ensure 'patient_id' is in the correct format for submission
test_data['patient_id'] = test_data['patient_id'].astype(str)

In [18]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [19]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)


In [20]:
# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [22]:
# Display the model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        40,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,769 (190.50 KB)

 Trainable params: 48,769 (190.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Fit the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=15, batch_size=32, verbose=1)


Epoch 1/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 13784.5664 - val_loss: 6973.2524
Epoch 2/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6959.0610 - val_loss: 6905.3511
Epoch 3/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6834.1621 - val_loss: 6923.5049
Epoch 4/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6794.9966 - val_loss: 6846.4712
Epoch 5/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6673.6333 - val_loss: 6837.0947
Epoch 6/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6835.0684 - val_loss: 6854.7476
Epoch 7/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6570.3120 - val_loss: 6926.0229
Epoch 8/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6712.3291 - val_loss: 6898.3091
Epoch 9/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6530.9570 - val_loss: 6835.3130
Epoch 10/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6328.0317 - val_loss: 6877.8442
Epoch 11/15
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6536.9385 - val_l

In [24]:
# Evaluate the model on validation data
val_predictions = model.predict(X_val_scaled)
val_rmse = mean_squared_error(y_val, val_predictions, squared=False)
print("Validation RMSE:", val_rmse)


83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Validation RMSE: 83.0245607812706


In [25]:
# Predict on the test set
test_predictions = model.predict(scaler.transform(test_data))



177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [26]:
# Prepare submission file
submission = pd.DataFrame({
    'patient_id': test_data['patient_id'],
    'metastatic_diagnosis_period': test_predictions.flatten() # Flatten predictions to convert to 1D array
})

# Save submission file
submission.to_csv('dl_submission.csv', index=False)